In [78]:
# code to convert big fat csv to reduced/required csv

import pandas as pd
import findspark
from pyspark.sql import SparkSession

findspark.init("/usr/local/spark")
# Build the SparkSession
spark = SparkSession.builder    .master("local")    .appName(
    "Crime Score")    .config("spark.executor.memory", "1gb")    .getOrCreate()
sc = spark.sparkContext

df = None
# pandas_df = pd.read_csv('rp_data.csv')  
df = (spark.read.format("csv").options(header="true")
    .load("rp_data_la.csv"))


In [79]:
from pyspark.sql.functions import split
split_col = split(df['Date'], '/')
df = df.withColumn('Year', split_col.getItem(2))
df = df.withColumn('Month', split_col.getItem(0))
df.show()
# sdf = df.groupBy(['Zipcode','Year','Month','Crime_Type']).count()
# sdf.toPandas().to_csv('grouped_la.csv')
# df = df.withColumn('Month', split_col.getItem(1))
# sdf = df.groupBy(['Zipcode','Year','Month','Primary Type']).count()
# df.toPandas().to_csv('grouped.csv')

+---------+----------+--------------------+--------------------+--------------------+-------+------------+-------------------+-----------+----+-----+
|  Case_ID|      Date|   Crime_Description|Location_Description|         Geolocation|Zipcode|Rental_Price|         Crime_Type|       City|Year|Month|
+---------+----------+--------------------+--------------------+--------------------+-------+------------+-------------------+-----------+----+-----+
|161816333|08/18/2016|VANDALISM - FELON...|VEHICLE, PASSENGE...|(33.9328, -118.2621)|90061.0|      2090.0|    CRIMINAL DAMAGE|Los Angeles|2016|   08|
|161816334|08/18/2016|    VEHICLE - STOLEN|              STREET|(33.9581, -118.2651)|90003.0|      2047.0|MOTOR VEHICLE THEFT|Los Angeles|2016|   08|
|161816335|08/18/2016|BURGLARY FROM VEH...|PROJECT/TENEMENT/...| (33.9573, -118.263)|90003.0|      2047.0|           BURGLARY|Los Angeles|2016|   08|
|161816337|08/18/2016|VANDALISM - MISDE...|            DRIVEWAY|(33.9292, -118.2695)|90061.0|      2

In [72]:
df.select("Year").distinct().count()

10

In [80]:
crimeSampleScore = {
    'NARCOTICS': 16,
    'BATTERY': 4,
    'THEFT': 9,
    'CRIMINAL DAMAGE': 9,
    'OTHER OFFENSE': 1,
    'ASSAULT': 36,
    'ROBBERY': 25,
    'BURGLARY': 16,
    'MOTOR VEHICLE THEFT': 9,
    'DECEPTIVE PRACTICE': 4,
    'PROSTITUTION': 16,
    'CRIMINAL TRESPASS': 4,
    'WEAPONS VIOLATION': 16,
    'PUBLIC PEACE VIOLATION': 9,
    'GAMBLING': 4,
    'OFFENSE INVOLVING CHILDREN': 16,
    'CRIM SEXUAL ASSAULT': 16,
    'INTERFERENCE WITH PUBLIC OFFICER': 4,
    'SEX OFFENSE': 9,
    'HOMICIDE': 49,
    'ARSON': 36,
    'LIQUOR LAW VIOLATION': 9,
    'KIDNAPPING': 36,
    'INTIMIDATION': 4,
    'STALKING': 9,
    'OBSCENITY': 16,
    'CONCEALED CARRY LICENSE VIOLATION': 4,
    'PUBLIC INDECENCY': 9,
    'HUMAN TRAFFICKING': 25,
    'NON-CRIMINAL': 1,
    'OTHER NARCOTIC VIOLATION': 4,
    'NON - CRIMINAL': 1,
    'RITUALISM': 1,
    'NON-CRIMINAL (SUBJECT SPECIFIED)': 1,
    'DOMESTIC VIOLENCE': 36,
}
sampleCrimeByCommunityBody = {
    "size": 0,
    "aggs": {
        "yearWise": {
            "terms": {"field": "Year",
                      "size": 20,
                      "order": {"_key": "asc"}
                      }, "aggs": {
                "monthWise": {
                    "terms": {
                        "field": "Month",
                        "size": 400,
                         "order": {"_key": "asc"}
                    }, "aggs": {
                        "zipCodeWise": {
                            "terms": {
                                "field": "Zipcode",
                                "size": 300
                            }, "aggs": {
                                "primaryTypeWise": {
                                    "terms": {
                                        "field": "Crime_Type",
                                        "size": 100
                                    }, "aggs": {
                                        "total": {"sum": {"field": "count"}}
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }
}



In [81]:
# get crime data from elastic search
# get crime data from elastic search
import json
import requests
import statistics
import math

def getCrimesByCommunity(bodyContent):

    api_url = 'http://localhost:9200/crime_la/_search?pretty'
    headers = { 'Content-Type': 'application/json'}
    response = requests.get(api_url, data=json.dumps(bodyContent), headers = headers)
    if response.status_code == 200:
        return response.json()
    else:
        return None

crimeDataByZipcode = getCrimesByCommunity(sampleCrimeByCommunityBody)
deprecationFactor = 0.50
# setup weights for each crime type
# voilent crimes : Homicide, Rape and Sexual Assault Robbery, Assault, Stalking/intimidation
# property crimes : Burglary, Larceny/theft, Motor vehicle theft, Cybercrime - Electronic crime, Identity theft
# Drugs & Crime
# Gangs, Hate Crime, Human Trafficking/Trafficking in Persons, Cybercrime, Identity Theft, Weapon Use
# divide crime by communityId
scorePerCrime = crimeSampleScore
sumOfSampleScores = sum(scorePerCrime.values())
scalingFactor = (1000/sumOfSampleScores)
scorePerCrime = dict(map(lambda kv: (kv[0], kv[1]*scalingFactor), scorePerCrime.items()))

# calculate total crime count per type
previousCalculation = {}
yearMap = {}
for yearWise in crimeDataByZipcode['aggregations']['yearWise']['buckets']:
    monthMap = {}
    for monthWise in yearWise['monthWise']['buckets']:
        # find max & min among all zipcodes for this month
        crimeTypeMaxVal = {}
        crimeTypeMinVal = {}
        for zipCodeWise in monthWise['zipCodeWise']['buckets']:
            
            for crimeType in zipCodeWise['primaryTypeWise']['buckets']:
                if crimeType['key'] in crimeTypeMaxVal:
                    if crimeType['total']['value'] > crimeTypeMaxVal[crimeType['key']]:
                        crimeTypeMaxVal[crimeType['key']] = crimeType['total']['value']
                    if crimeType['total']['value'] < crimeTypeMinVal[crimeType['key']]:
                        crimeTypeMinVal[crimeType['key']] = crimeType['total']['value']
                else:
                    crimeTypeMaxVal[crimeType['key']]=crimeType['total']['value']
                    crimeTypeMinVal[crimeType['key']]=crimeType['total']['value']
        # calculate score for each community
        # score is the weighted score (min/max scaling) for each crime type (lookup constants.py for score values per crime type)
        scorePerZipcode = {}
        for zipCodeWise in monthWise['zipCodeWise']['buckets']:
            currentZipcodeMap = {}
            score = sum(map(lambda x: (((x['total']['value'] - crimeTypeMinVal[x['key']])/(crimeTypeMaxVal[x['key']] - crimeTypeMinVal[x['key']]))*scorePerCrime[x['key']] if ((crimeTypeMaxVal[x['key']] - crimeTypeMinVal[x['key']]) > 0) else 1),zipCodeWise['primaryTypeWise']['buckets']))
            scorePerZipcode[str(zipCodeWise['key'])] = score
        # accounting previousCalculation in calculating the new scores
        for key, value in scorePerZipcode.items():
            if key in previousCalculation:
                scorePerZipcode[key] = deprecationFactor*previousCalculation[key] + (1.0 - deprecationFactor)*value
        for key, value in previousCalculation.items():
            if key not in scorePerZipcode:
                scorePerZipcode[key] = value
        previousCalculation = scorePerZipcode
        monthKey = ''
        if monthWise['key'] < 10:
            monthKey=monthKey+'0'
        monthKey+=str(monthWise['key'])
        monthMap[monthKey] = scorePerZipcode
    yearMap[str(yearWise['key'])] = monthMap
# map(lambda x: ((x - meanCrimeScore)/stdevCrimeScore),scorePerCommunity)
# create a new index in elastic search to insert score for each communityId
print(yearMap)

{'2010': {'01': {'90003.0': 373.9811518505382, '90008.0': 156.4447084594402, '90031.0': 132.85594679166888, '90006.0': 101.25761713226812, '90011.0': 552.7893538033987, '91306.0': 190.34728056368547, '91343.0': 133.82268967000923, '91402.0': 228.56956746696764, '90019.0': 97.48153725764313, '90029.0': 250.11888461935249, '90057.0': 268.04057511519966, '90062.0': 306.0203800823816, '91325.0': 109.85902826880836, '91331.0': 189.52431336933395, '90002.0': 276.64473123429696, '90004.0': 140.15840550361716, '90018.0': 238.82759330722766, '90025.0': 74.6068526920173, '90038.0': 81.66513912957015, '90043.0': 117.57695720337786, '90731.0': 147.3835502403304, '91303.0': 150.29943636788653, '91342.0': 77.91214835075309, '91367.0': 97.16625323942883, '91401.0': 141.16784974714764, '91405.0': 199.84553372193332, '91601.0': 82.09857295748031, '91606.0': 218.4989918745094, '90005.0': 70.20113752450028, '90007.0': 170.78003169183154, '90013.0': 40.789348914363835, '90016.0': 108.02367095394477, '9002

In [82]:
from pyspark.sql.functions import udf,col
from pyspark.sql.types import FloatType

def tryconvert(year, month, zipcode):
    try:
        return yearMap[year][month][zipcode]
    except KeyError:
        return 0.0
function = udf(lambda year, month, zipcode : tryconvert(year,month,zipcode), FloatType())
df = df.withColumn('Neighborhood_Score', function(col('Year'), col('Month'), col('Zipcode')))

In [83]:
columns_to_drop = ['Year', 'Month']
df = df.drop(*columns_to_drop)

In [85]:
from pyspark.sql import DataFrameWriter
# df.show(235)
# df.printSchema()
df.coalesce(1).write.csv(path="./updated_rental_crime_la.csv", mode="append", header=True,sep=',')
# panda_dataframe = df.toPandas()
# panda_dataframe.to_csv('updated_rental_crime_la.csv.csv.gz'
#          , sep=','
#          , header=True
#          , index=False
#          , chunksize=100000
#          , compression='gzip'
#          , encoding='utf-8')

In [86]:
spark.stop()